# Model Selection

In [1]:
import pandas as pd
from lazypredict.Supervised import LazyRegressor
from sklearn.model_selection import train_test_split
import warnings

warnings.filterwarnings('ignore')

In [2]:
df = pd.read_parquet('datasets/featured/featured.parquet')

In [3]:
df.head()

,MSSubClass,LotFrontage,LotArea,Street,Utilities,Condition2,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofMatl,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,CentralAir,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,...,BsmtFinType2_Rare,BsmtFinType2_Rec,BsmtFinType2_Unf,HeatingQC_Fa,HeatingQC_Gd,HeatingQC_Rare,HeatingQC_TA,Electrical_FuseF,Electrical_Rare,Electrical_SBrkr,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_TA,Functional_Min2,Functional_Rare,Functional_Typ,GarageType_BuiltIn,GarageType_Detchd,GarageType_Rare,GarageFinish_RFn,GarageFinish_Unf,GarageQual_Rare,GarageQual_TA,GarageCond_Rare,GarageCond_TA,PavedDrive_P,PavedDrive_Y,SaleType_New,SaleType_Rare,SaleType_WD,SaleCondition_Normal,SaleCondition_Partial,SaleCondition_Rare,TotalSF,Season_Spring,Season_Summer,Season_Winter,QualityScore,PorchSF,LotCoverage
0,60.00,65.00,8450.00,0,0,0,7.00,5.00,2003.00,2003.00,0,196.00,706.00,0.00,150.00,856.00,0,1,856.00,854.00,0.00,1710.00,1.00,0.00,2.00,1.00,3.00,1.00,8.00,0.00,2003.00,2.00,548.00,0.00,61.00,0.00,0.00,0.00,0.00,0.00,...,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,1,0,1,0,1,0,0,1,1,0,0,2566.00,0,0,1,5.83,61.00,0.30
1,20.00,80.00,9600.00,0,0,0,6.00,8.00,1976.00,1976.00,0,0.00,978.00,0.00,284.00,1262.00,0,1,1262.00,0.00,0.00,1262.00,0.00,1.00,2.00,0.00,3.00,1.00,6.00,1.00,1976.00,2.00,460.00,298.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,1,0,1,0,1,0,0,1,1,0,0,2524.00,1,0,0,7.15,0.00,0.26
2,60.00,68.00,11250.00,0,0,0,7.00,5.00,2001.00,2002.00,0,162.00,486.00,0.00,434.00,920.00,0,1,920.00,866.00,0.00,1786.00,1.00,0.00,2.00,1.00,3.00,1.00,6.00,1.00,2001.00,2.00,608.00,0.00,42.00,0.00,0.00,0.00,0.00,0.00,...,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,1,0,1,0,1,0,0,1,1,0,0,2706.00,0,0,0,5.83,42.00,0.24
3,70.00,60.00,9550.00,0,0,0,7.00,5.00,1915.00,1970.00,0,0.00,216.00,0.00,540.00,756.00,0,1,961.00,756.00,0.00,1717.00,1.00,0.00,1.00,0.00,3.00,1.00,7.00,1.00,1998.00,3.00,642.00,0.00,35.00,272.00,0.00,0.00,0.00,0.00,...,0,0,1,0,1,0,0,0,0,1,0,1,0,0,0,1,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0,0,0,2473.00,0,0,1,5.83,307.00,0.26
4,60.00,84.00,14260.00,0,0,0,8.00,5.00,2000.00,2000.00,0,350.00,655.00,0.00,490.00,1145.00,0,1,1145.00,1053.00,0.00,2198.00,1.00,0.00,2.00,1.00,4.00,1.00,9.00,1.00,2000.00,3.00,836.00,192.00,84.00,0.00,0.00,0.00,0.00,0.00,...,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,1,0,1,0,1,0,0,1,1,0,0,3343.00,0,0,1,6.39,84.00,0.23


## Models

In [4]:
X = df.drop('SalePrice', axis=1)
y = df[['SalePrice']]

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7)

In [6]:
reg_models = LazyRegressor(verbose=0, ignore_warnings=False, custom_metric=None)

In [7]:
models, predictions = reg_models.fit(X_train, X_test, y_train, y_test)

100%|██████████| 42/42 [00:46<00:00,  1.10s/it]


In [8]:
models

,Adjusted R-Squared,R-Squared,RMSE,Time Taken
Model,,,,
PoissonRegressor,0.84,0.94,19617.61,0.05
GradientBoostingRegressor,0.80,0.92,21566.18,0.87
ExtraTreesRegressor,0.80,0.92,21715.55,1.91
GammaRegressor,0.80,0.92,21762.65,0.03
XGBRegressor,0.80,0.92,21819.51,0.44
PassiveAggressiveRegressor,0.79,0.92,21958.76,0.38
RandomForestRegressor,0.79,0.92,22125.69,2.75
ElasticNet,0.79,0.92,22245.99,0.04
HuberRegressor,0.79,0.92,22275.17,0.17


## Create Model

In [6]:
from xgboost import XGBRegressor
from sklearn.model_selection import cross_validate
from sklearn import metrics

import numpy as np

## XGBRegressor

In [79]:
scores = []
for c in range(1, 11):
    reg = XGBRegressor()
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=c)
    reg.fit(X_train, y_train)
    scores.append(reg.score(X_test, y_test))

In [80]:
np.mean(scores)

0.8582174720780869

## GradientBoostingRegressor

In [32]:
from sklearn.ensemble import GradientBoostingRegressor

In [81]:
scores = []
for c in range(1, 11):
    reg = GradientBoostingRegressor()
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=c)
    reg.fit(X_train, y_train)
    scores.append(reg.score(X_test, y_test))

In [82]:
np.mean(scores)

0.8621504931129881

## LinearRegression

In [40]:
from sklearn.linear_model import LinearRegression

In [85]:
scores = []
for c in range(1, 11):
    reg = LinearRegression()
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=c)
    reg.fit(X_train, y_train)
    scores.append(reg.score(X_test, y_test))

In [86]:
np.mean(scores)

0.822595685125554

## ExtraTreesRegressor

In [76]:
from sklearn.ensemble import ExtraTreesRegressor

In [87]:
scores = []
for c in range(1, 11):
    reg = ExtraTreesRegressor()
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=c)
    reg.fit(X_train, y_train)
    scores.append(reg.score(X_test, y_test))

In [88]:
np.mean(scores)

0.8622426797123129

Decided to **XGBRegressor**.

# XGBRegressor

In [90]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7)

In [91]:
reg = XGBRegressor()

In [92]:
reg.fit(X_train, y_train)
reg.score(X_test, y_test)

0.9210399455352938

In [93]:
reg.get_params()

{'objective': 'reg:squarederror',
 'base_score': None,
 'booster': None,
 'callbacks': None,
 'colsample_bylevel': None,
 'colsample_bynode': None,
 'colsample_bytree': None,
 'early_stopping_rounds': None,
 'enable_categorical': False,
 'eval_metric': None,
 'feature_types': None,
 'gamma': None,
 'gpu_id': None,
 'grow_policy': None,
 'importance_type': None,
 'interaction_constraints': None,
 'learning_rate': None,
 'max_bin': None,
 'max_cat_threshold': None,
 'max_cat_to_onehot': None,
 'max_delta_step': None,
 'max_depth': None,
 'max_leaves': None,
 'min_child_weight': None,
 'missing': nan,
 'monotone_constraints': None,
 'n_estimators': 100,
 'n_jobs': None,
 'num_parallel_tree': None,
 'predictor': None,
 'random_state': None,
 'reg_alpha': None,
 'reg_lambda': None,
 'sampling_method': None,
 'scale_pos_weight': None,
 'subsample': None,
 'tree_method': None,
 'validate_parameters': None,
 'verbosity': None}

## Hyperparameter Tunning

In [99]:
from sklearn.model_selection import GridSearchCV

In [101]:
hyperparameters = {
    'n_estimators': [100, 400, 800],
    'max_depth': [None, 3, 6, 9],
    'learning_rate': [None, 0.1, 0.01, 0.001],
    'min_child_weight': [None, 1, 10, 100],
    "colsample_bytree": [None, 0.5, 0.7, 1]
    }

In [102]:
xgb_best_grid = GridSearchCV(reg, hyperparameters, cv=5, n_jobs=-1, verbose=True).fit(X, y)

Fitting 5 folds for each of 768 candidates, totalling 3840 fits


In [103]:
xgb_best_grid.best_score_

0.892971480697225

In [104]:
xgb_best_grid.best_params_

{'colsample_bytree': 0.5,
 'learning_rate': 0.1,
 'max_depth': 3,
 'min_child_weight': None,
 'n_estimators': 400}

In [105]:
hyperparameters = {
    'n_estimators': [200, 300, 400, 500, 600],
    'max_depth': [2, 3, 4, 5],
    'learning_rate': [0.1],
    "colsample_bytree": [0.4, 0.5, 0.6]
    }

In [106]:
xgb_best_grid = GridSearchCV(reg, hyperparameters, cv=5, n_jobs=-1, verbose=True).fit(X, y)

Fitting 5 folds for each of 60 candidates, totalling 300 fits


In [107]:
xgb_best_grid.best_score_

0.8939164880193001

In [108]:
xgb_best_grid.best_params_

{'colsample_bytree': 0.4,
 'learning_rate': 0.1,
 'max_depth': 3,
 'n_estimators': 400}

In [109]:
hyperparameters = {
    'n_estimators': [400],
    'max_depth': [3],
    'learning_rate': [0.1],
    "colsample_bytree": [0.4],
    'min_child_weight': [None, 1, 5, 10],
    'gamma': [None, 0.5, 1, 1.5, 2, 5],
    'subsample': [None, 0.6, 0.8, 1.0],
    }

In [110]:
xgb_best_grid = GridSearchCV(reg, hyperparameters, cv=5, n_jobs=-1, verbose=True).fit(X, y)

Fitting 5 folds for each of 96 candidates, totalling 480 fits


In [111]:
xgb_best_grid.best_score_

0.8939164880193001

In [112]:
xgb_best_grid.best_params_

{'colsample_bytree': 0.4,
 'gamma': None,
 'learning_rate': 0.1,
 'max_depth': 3,
 'min_child_weight': None,
 'n_estimators': 400,
 'subsample': None}

In [119]:
hyperparameters ={
 'colsample_bytree': [0.4],
 'gamma': [None],
 'learning_rate': [0.1, 0.15, 0.2],
 'max_depth': [3],
 'min_child_weight': [None],
 'n_estimators': [400],
 'subsample': [None],
 'booster': [None, 'gbtree','gblinear']}

In [120]:
xgb_best_grid = GridSearchCV(reg, hyperparameters, cv=5, n_jobs=-1, verbose=True).fit(X, y)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


In [121]:
xgb_best_grid.best_score_

0.8939164880193001

In [122]:
xgb_best_grid.best_params_

{'booster': None,
 'colsample_bytree': 0.4,
 'gamma': None,
 'learning_rate': 0.1,
 'max_depth': 3,
 'min_child_weight': None,
 'n_estimators': 400,
 'subsample': None}

## Final Model

In [123]:
xgboost_final = reg.set_params(**xgb_best_grid.best_params_).fit(X, y)

In [124]:
cv_results = cross_validate(xgboost_final, X, y, cv=5, scoring=["r2"])

In [127]:
cv_results['test_r2'].mean()

0.8939164880193001

In [128]:
xgboost_final.save_model('model')